# Load modules, define model and detector

In [2]:
import torch
import torchvision.transforms as T
from PIL import Image

# standard PyTorch mean-std input image normalization
transform = T.Compose([
  T.Resize(800),
  T.ToTensor(),
  T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def detect(im, model, transform):
  # define help functions
  # for output bounding box post-processing
  def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

  def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

  # mean-std normalize the input image (batch-size: 1)
  img = transform(im).unsqueeze(0)

  # propagate through the model
  outputs = model(img)

  # keep only predictions with 0.7+ confidence
  probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]
  keep = probas.max(-1).values > 0.7

  # convert boxes from [0; 1] to image scales
  bboxes_scaled = rescale_bboxes(outputs['pred_boxes'][0, keep], im.size)
  return probas[keep], bboxes_scaled

# import model from pytorch model hub
model = torch.hub.load('facebookresearch/detr', 'detr_resnet50', pretrained=True)

Using cache found in /home/max/.cache/torch/hub/facebookresearch_detr_master


# Inference

load image

In [3]:
im = Image.open('res_test/test.jpg')

run inference

In [4]:
scores, boxes = detect(im, model, transform)

In [14]:
scores

tensor([[5.7541e-11, 8.4872e-01, 9.2338e-08, 1.7895e-06, 8.5537e-08, 1.1305e-06,
         1.3379e-06, 1.7005e-06, 3.1960e-06, 2.4768e-06, 1.2717e-07, 7.7536e-08,
         1.7352e-11, 1.5053e-07, 6.1385e-09, 3.0014e-06, 4.7095e-07, 1.9473e-07,
         5.0021e-06, 7.3698e-06, 8.7648e-08, 2.3988e-07, 6.9246e-06, 1.1698e-06,
         4.2513e-08, 1.1801e-06, 3.0026e-11, 4.2710e-06, 4.5627e-06, 3.4581e-11,
         3.1758e-11, 1.6620e-06, 7.9261e-06, 2.7949e-07, 5.8136e-08, 1.5599e-06,
         5.4041e-07, 1.6361e-06, 1.9244e-06, 9.9549e-07, 1.4967e-07, 2.9710e-07,
         4.7768e-06, 3.2777e-07, 1.4398e-06, 3.3702e-11, 2.1826e-07, 1.6778e-07,
         1.0074e-07, 3.8919e-07, 2.3661e-07, 1.0826e-07, 8.7650e-07, 4.7793e-07,
         1.2499e-08, 3.7259e-09, 6.2353e-07, 2.9604e-07, 9.6997e-08, 5.6025e-07,
         4.1215e-08, 7.1220e-07, 5.3505e-06, 1.3510e-05, 1.0796e-06, 5.0603e-06,
         3.2748e-11, 1.3011e-06, 4.8524e-11, 5.3775e-11, 2.2178e-07, 3.8750e-11,
         2.2420e-06, 7.8460e